In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pymc3 import Model, Normal, HalfNormal, find_MAP, NUTS, sample
from pymc3 import traceplot, summary
from scipy import optimize
import pandas as pd
from pymc3 import Exponential, StudentT, Deterministic
from pymc3.distributions.timeseries import GaussianRandomWalk
from theano.tensor import exp

/home/junpenglao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
np.random.seed(123)
alpha, sigma = 1, 1
beta = [1, 2.5]
size = 100
X1 = np.linspace(0, 1, size)
X2 = np.linspace(0,.2, size)
Y = alpha + beta[0]*X1 + beta[1]*X2 + np.random.randn(size)*sigma

returns = pd.read_csv('SP500.csv', index_col=0, parse_dates=True, sep='delimiter', header=None, engine='python')

FileNotFoundError: [Errno 2] No such file or directory: 'SP500.csv'

In [ ]:
basic_model = Model()
with basic_model:
alpha = Normal('alpha', mu=0, sd=10)
beta = Normal('beta', mu=0, sd=10, shape=2)
sigma = HalfNormal('sigma', sd=1)
mu = alpha + beta[0]X1 + beta[1]X2
Y_obs = Normal('Y_obs', mu=mu, sd=sigma, observed=Y)
with Model() as sp500_model:
nu = Exponential('nu', 1./10, testval=5.)
sigma = Exponential('sigma', 1./.02, testval=.1)
s = GaussianRandomWalk('s', sigma-2, shape=len(returns))
volatility_process = Deterministic('volatility_process', exp(-2*s))
r = StudentT('r', nu, lam=1/volatility_process)
with sp500_model:
start = find_MAP(vars=[s], fmin=optimize.fmin_l_bfgs_b)
step = NUTS(scaling=start)
trace = sample(100, step, progressbar=False)